In [127]:
import requests

In [9]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import sqlalchemy as sql
import sqlite3
pd.options.display.max_columns = None

Populating the interactive namespace from numpy and matplotlib


In [7]:
def get_put_db(db_name):
    conn = sqlite3.connect(db_name)
    weekly_dfs = {}
    for week in range(1,34): 
        data = pd.read_csv(f'data/GW{week}.csv') 
        weekly_dfs[week] = data 

    #dding the next gameweeks total points to current week statistic for each player
    weekly = {}
    for week in weekly_dfs: 
        #we cannot add gw 33 here because we do not have the reults for gw 32 to predict and score against yet as gw32 is not played yet
        if week != 33: 
            df = pd.merge(weekly_dfs[week], weekly_dfs[week+1][['name', 'Pts']], on='name', how='inner')  
            weekly[week] = df

    # add all the data together.         
    full_df = pd.concat([x[1] for x in list(weekly.items())], axis=0)
    full_df.set_index('name', inplace=True)
    
    full_df.to_sql('stats', con=conn, if_exists='replace')
    
    
get_put_db('./data/fantasy.db')

In [38]:
engine = sql.create_engine("sqlite:///fantasy.db")
engine.table_names()
df = pd.read_sql('select element, was_home,team_h_score, team_a_score, minutes,goals_scored, assists, clean_sheets,total_points, total_points_y from all_weeks', engine) 


In [73]:
import pickle as pkl

#     df['log_y']= np.log10((df['total_points_y'])+5)
#     df.dropna(inplace=True)


X=df.drop(['total_points_y', 'element'], axis=1)
y = df['total_points_y']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=200) 

# fit our model and score it on the test set
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

print(f"""
train score: {lr_model.score(x_train, y_train):%}
test score: {lr_model.score(x_test, y_test):%}
""")
preds = lr_model.predict(x_test) 

#     pred_y = (((10**log_pred_y)-5))

# report results

print('Model Error')
print('MSE: ', mean_squared_error(y_test, preds)) 
print('MAE: ', mean_absolute_error(y_test, preds), '\n')   

print('Feature coefficient results: \n')
for feature, coef in zip(X.columns, lr_model.coef_):
    print(feature, ':', f'{coef:.2f}')

with open("models/lr.pkl", "wb") as f:
    pkl.dump(lr_model, f)




train score: 23.175826%
test score: 24.028688%

Model Error
MSE:  4.536783952002645
MAE:  1.2446540862421105 

Feature coefficient results: 

was_home : 0.01
team_h_score : -0.02
team_a_score : 0.01
minutes : 0.03
goals_scored : 0.19
assists : 0.05
clean_sheets : -0.09
total_points : 0.09


In [74]:
x_ = [1,6,2,90, 2, 1,0,17]

In [79]:
with open("models/lr.pkl", "rb") as f:
    lr_model2 = pkl.load(f)

In [94]:
lr_model2.predict([x_])

array([4.5117205])

In [119]:
web_name = "Fernandes"
first_name = "Bruno Miguel"
round_ = 33 
querry = f'''
SELECT a.was_home, a.team_h_score,a.team_a_score, a.minutes, a.goals_scored, a.assists, a.clean_sheets, a.total_points
FROM  elements as e
JOIN all_weeks as a
ON e.id=a.element
WHERE e.web_name = "{web_name}" and e.first_name="{first_name}"  AND a.round = {round_}
ORDER by a.round
'''
engine1 = sql.create_engine("sqlite:///fantasy.db")
var = pd.read_sql(querry, engine1).values
answer = lr_model2.predict(var)[0]

In [126]:
name_q = '''
SELECT web_name
FROM  elements 
'''
pd.read_sql(name_q, engine1).values


array([['Özil'],
       ['Sokratis'],
       ['David Luiz'],
       ['Aubameyang'],
       ['Cédric'],
       ['Lacazette'],
       ['Mustafi'],
       ['Leno'],
       ['Xhaka'],
       ['Marí'],
       ['Bellerín'],
       ['Chambers'],
       ['Kolasinac'],
       ['Holding'],
       ['Tierney'],
       ['Pépé'],
       ['Torreira'],
       ['Nelson'],
       ['Nketiah'],
       ['Smith Rowe'],
       ['Saka'],
       ['Guendouzi'],
       ['Martinelli'],
       ['Saliba'],
       ['Ryan'],
       ['Willian'],
       ['Gabriel'],
       ['Ceballos'],
       ['Elneny'],
       ['Macey'],
       ['Rúnarsson'],
       ['Partey'],
       ['Ødegaard'],
       ['Azeez'],
       ['Martínez'],
       ['Heaton'],
       ['El Mohamady'],
       ['Lansbury'],
       ['Taylor'],
       ['Steer'],
       ['Hourihane'],
       ['Jota'],
       ['Nyland'],
       ['Engels'],
       ['Grealish'],
       ['McGinn'],
       ['Hause'],
       ['Trézéguet'],
       ['Mings'],
       ['Targett'],
      

We can see from above that we are still overfitting. Moreover, we can also see even our best MSE and MAE are still quite apart(mse > mae)
which indicates the presence of outliers in the data